In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from joblib import Parallel, delayed

# Grid parameters
grid_h, grid_w = 50, 50
radius = 1
n_points = 8 * radius

def extract_features_from_cell(cell):
    features = []
    # Intensity
    features.append(np.mean(cell))
    features.append(np.std(cell))
    
    # LBP
    lbp = local_binary_pattern(cell, n_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points+3), density=True)
    features.extend(hist)
    
    # Edge density
    edges = cv2.Canny(cell, 50, 150)
    features.append(np.sum(edges > 0) / cell.size)
    
    return np.array(features)

def extract_features_from_image(img):
    features_list = []
    h, w = img.shape
    for y in range(0, h, grid_h):
        for x in range(0, w, grid_w):
            cell = img[y:y+grid_h, x:x+grid_w]
            if cell.shape[0] != grid_h or cell.shape[1] != grid_w:
                cell = cv2.resize(cell, (grid_w, grid_h))
            features_list.append(extract_features_from_cell(cell))
    return np.vstack(features_list)

def load_and_process_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, 0
    img = cv2.resize(img, (800, 600))
    feats = extract_features_from_image(img)
    return feats, feats.shape[0]

def load_images_parallel(image_folder, n_jobs=-1):
    paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
    results = Parallel(n_jobs=n_jobs)(delayed(load_and_process_image)(p) for p in paths)
    
    all_features = []
    grid_indices = []
    for idx, (feats, n_grids) in enumerate(results):
        if feats is not None:
            all_features.append(feats)
            grid_indices.append((idx, n_grids))
    if all_features:
        all_features = np.vstack(all_features)
    else:
        all_features = np.array([])
    return all_features, grid_indices

# Main pipeline
if __name__ == "__main__":
    image_folder = "images"
    all_features, grid_indices = load_images_parallel(image_folder, n_jobs=-1)

    # Scale features
    scaler = StandardScaler()
    all_features_scaled = scaler.fit_transform(all_features)

    # Unsupervised detection
    iso = IsolationForest(contamination=0.1, random_state=42, n_jobs=-1)
    iso.fit(all_features_scaled)
    preds = iso.predict(all_features_scaled)  # -1 = anomaly (wildlife), 1 = normal

    # Map predictions back to images
    start = 0
    for img_idx, n_grids in grid_indices:
        grid_pred = preds[start:start+n_grids]
        start += n_grids
        num_rows = 600 // grid_h
        num_cols = 800 // grid_w
        grid_map = (grid_pred == -1).astype(int).reshape((num_rows, num_cols))
        print(f"Image {img_idx} grid map (1 = wildlife):")
        print(grid_map)